In [14]:
from dotenv import load_dotenv
from polygon import RESTClient
from datetime import datetime,timezone
import os

In [15]:
import pandas as pd

In [16]:
import numpy as np

In [17]:

load_dotenv("../guidelight-api/.env")
token = os.getenv("POLYGON_TOKEN")

In [18]:

client = RESTClient(api_key=token)

In [19]:
ticker = "AAPL"

# List Aggregates (Bars)
aggs = []
for a in client.list_aggs(ticker=ticker, multiplier=1, timespan="minute", from_="2019-01-01", to="2023-12-30", limit=50000):
    aggs.append(a)

print(len(aggs))



925448


In [20]:
aggs[-1]

Agg(open=192.1505, high=192.1505, low=192.1501, close=192.1501, volume=2147, vwap=192.1512, timestamp=1703897940000, transactions=12, otc=None)

In [21]:
#unix millisecond timestamp to datetime
# from pytz import timezone

# Convert timestamp to datetime in UTC timezone
def ts_to_dt(timestamp):
    return datetime.fromtimestamp(timestamp/1000, tz=timezone.utc)

timestamps = np.vectorize(ts_to_dt)

# Example usage
a = np.array([a.timestamp for a in aggs])
times = timestamps(a)
aapl_aggs = pd.DataFrame({"day": [time.strftime("%Y-%m-%d") for time in times],
                            "timestamp": [time.strftime("%Y-%m-%d, %H:%M") for time in times],
                           "close": [a.close for a in aggs], 
                           "volume": [a.volume for a in aggs],
                           "low": [a.low for a in aggs],
                           "high": [a.high for a in aggs]})



In [22]:
from heston_param import *

In [23]:
aapl_aggs.groupby("day")

In [24]:
daily_aggs =  [aapl_aggs.groupby('day').get_group(x) for x in aapl_aggs['day'].unique()]

In [25]:
hist_voilatilities = torch.empty(len(daily_aggs))
for i, day in enumerate(daily_aggs):
	hist_voilatilities[i] = estimate_historical_volatility(day['close'].values)

# calculate the historical volatility for each day



# historical_volatility
print(hist_voilatilities)

tensor([0.0097, 0.0100, 0.0094,  ..., 0.0055, 0.0066, 0.0030])


In [30]:
print(calibrate_daily_parameters(hist_voilatilities[0], 0.1, daily_aggs[0]["close"].values, 0.0237, 390, 3))

KeyboardInterrupt: 